# Hard Code Naive Bayes Classifier

## Import thư viện

In [1]:
import pandas as pd
import numpy as np

## Load và tiền xử lý data

In [2]:
df = pd.read_csv('Dataset.csv', header=None)
# Sửa tên column
df.rename(columns={0:'class',1:'age',2:'gender',3:'survived'},inplace=True)
# Xuất ra 5 dòng đầu tiên
df.head()

,class,age,gender,survived
0,1st,adult,male,yes
1,1st,adult,male,yes
2,1st,adult,male,yes
3,1st,adult,male,yes
4,1st,adult,male,yes


## Xây dựng công cụ Naive Bayes Classifier

In [3]:
def train_test_split(df, test_size = 0.2):
    # Shuffle tập data
    newdf = df.iloc[np.random.permutation(len(df))].reset_index(drop=True)
    # Số lượng train
    train_amount = int(np.ceil(len(newdf)*(1-test_size)))
    df_train = newdf.iloc[:train_amount,:]
    df_test = newdf.iloc[train_amount:,:].reset_index(drop=True)
    return df_train, df_test
    
class Binary_Naive_Bayes_Classifier:
    def __init__(self, thresh = 0.5, pos_label=None):
        self.training_dict = {}
        self.thresh = thresh
        # Các giá trị để đánh giá model
        self.pos_label = pos_label # nhãn positive để tính precision/recall(yes/no)
        self.train_acc = None # Accuracy
        self.train_precision = None # Precision
        self.train_recall = None # Recall
        self.train_f1_score = None # F1 Score
    # Fit data để chuyển thành model
    def fit(self,df_train,print_acc=True):
        df = df_train.copy()
        self.label_name = df_train.columns[-1]
        # Tìm percentage của từng labels (yes/no)
        labelname1 = df.iloc[:,-1].value_counts().keys()[0]
        labelpercent1 =  df.iloc[:,-1].value_counts()[0]/len(df)
        labelname2 = df.iloc[:,-1].value_counts().keys()[1]
        labelpercent2 = 1 - labelpercent1
        self.training_dict['labels'] = {labelname1:labelpercent1,labelname2:labelpercent2}
        features_dict = {}
        for column in df.columns[:-1]:
            values_dict = {}
            for value in df[column].unique():
                labels_dict = {}
                for label in [labelname1,labelname2]:
                    # Tìm những hàng có giá trị đó (e.g 1st class)
                    temp_df_value = df[df[column] == value]
                    # Tìm những hàng có label đó (e.g 1st class label no)
                    temp_df_label = temp_df_value[temp_df_value.iloc[:,-1]==label] #df[df.iloc[:,-1]=='yes']
                    # Tính % được value (yes/no) theo giá trị đó trong column
                    label_percent = len(temp_df_label)/len(temp_df_value)
                    # Thêm % label đó vào label dict (e.g 1st class có 60% label no)
                    labels_dict[label] = label_percent
                # Thêm label_dict vào feature dict tương ứng với feature 'column' 
                # (e.g feature class có 1st 2nd, ứng với %yes/no)
                
                values_dict[value] = labels_dict
            features_dict[column] = values_dict
        self.training_dict['features'] = features_dict
        df_predicted = self.predict(df_train)
        self.train_acc = round(np.sum(df_predicted[self.label_name] == df_train[self.label_name])/len(df),2)
        if print_acc == True:
            print('Training accuracy: {} %'.format(self.train_acc*100))   
    def predict(self, df_input):
        df = df_input.copy()
        df[self.label_name] = np.nan
        label_list=[]
        # Xác suất xảy ra p(1) và p(2)
        p_label = []
        # Cập nhật p(1) và p(2)
        for label in self.training_dict['labels'].keys():
            p_label.append(self.training_dict['labels'][label])
        # Dự đoán với từng dòng trong df
        for i in range(len(df)):
            # Xác suất X xảy ra khi label 1/2 xảy ra
            p_X_label = []
            for label in self.training_dict['labels'].keys():
                temp_p_X_label = 1
                for feature in self.training_dict['features'].keys():
                    value = df.loc[i,feature]
                    temp_p_X_label *= self.training_dict['features'][feature][value][label]
                p_X_label.append(temp_p_X_label)
            # Tính p(X)
            p_X = p_label[0]*np.sum(p_X_label)
            predicted_percentage = p_X_label[0]*(p_label[0])/p_X
            predicted_label = None
            # Nếu > thresh% thì lấy label thứ 1, không thì lấy thứ 2
            if (predicted_percentage > self.thresh):
                predicted_label = list(self.training_dict['labels'].keys())[0]
            else:
                predicted_label = list(self.training_dict['labels'].keys())[1]
            # Cập nhật label vô dòng đó
            df.loc[i,self.label_name] = predicted_label
            df.loc[i,'confidence'] = predicted_percentage
        return df
    def evaluate(self, df_test, print_acc = True):
        test_acc = round(np.sum(myClassifier.predict(df_test)[self.label_name] == df_test[self.label_name])/len(df_test),2)
        if print_acc == True:
            print('Test accuracy: {} %'.format(test_acc*100))
        return test_acc

## Chạy thử model

In [7]:
np.random.seed(156)
# Chia data thành test và train với tỉ lệ test_size
df_train, df_test = train_test_split(df,test_size=0.25)
# Gọi class Naive Bayes để hỗ trợ, với thresh = 0.5 - ngưỡng kích hoạt yes/no
myClassifier = Binary_Naive_Bayes_Classifier(thresh=0.5)
# Build model dựa trên tập train và xuất ra độ chính xác trên tập training
myClassifier.fit(df_train)
# Dự đoán dựa trên tập data (có nhãn) và xuất ra độ chính xác trên tập test
_ = myClassifier.evaluate(df_test)

Training accuracy: 74.0 %
Test accuracy: 75.0 %


Như vậy model chúng ta có `test accuracy` là 75%, cũng khá cao nhưng không cao lắm. Chúng ta thử điều chỉnh lại tham số threshold xem kết quả có khả quan hơn không

# Hyperparameter tuning

Ở đây chúng ta sẽ tune siêu tham số `threshold`, là ngưỡng để quyết định 2 label yes/no

In [5]:
list_best_thresh = []
list_best_acc_thresh = []
# Sử dụng các random seed khác nhau để shuffle data
for seed in [1,2,3,4,5]:
    np.random.seed(seed)
    df_train, df_test = train_test_split(df,test_size=0.25)
    print("Random seed: ",seed)
    best_thresh = [0,0]
    for thresh in np.arange(0.4,0.9,0.05):
        thresh=thresh
        myClassifier = Binary_Naive_Bayes_Classifier(thresh=thresh)
        myClassifier.fit(df_train, print_acc = False)
        test_acc = myClassifier.evaluate(df_test, print_acc = False)
        if test_acc == best_thresh[0]:
            best_thresh.append(test_acc)
            continue
        if test_acc >= best_thresh[0]:
            best_thresh = []
            best_thresh.append(test_acc)
            best_thresh.append(thresh)

    print("Mean best threshold is {} with the accuracy of {}".format(np.mean(best_thresh[1:]),round(best_thresh[0],2)))
    list_best_thresh.append(best_thresh[0])
    list_best_acc_thresh.append(np.mean(best_thresh[1:]))
print("Best overall mean accuracy threshold is {} \
        with mean threshold {}".format(round(np.mean(list_best_thresh),2),round(np.mean(list_best_acc_thresh),2)))

Random seed:  1
Mean best threshold is 0.7350000000000001 with the accuracy of 0.78
Random seed:  2
Mean best threshold is 0.752 with the accuracy of 0.79
Random seed:  3
Mean best threshold is 0.7471428571428573 with the accuracy of 0.78
Random seed:  4
Mean best threshold is 0.752 with the accuracy of 0.79
Random seed:  5
Mean best threshold is 0.78 with the accuracy of 0.81
Best overall mean accuracy threshold is 0.79         with mean threshold 0.75


Như vậy chúng ta chọn `threshold = 0.75` vì có test_accuracy trung bình cao nhất

# Chạy lại model hoàn chỉnh

Lần này chúng ta sẽ sử dụng `threshold = 0.75` và thay đổi seed khác

In [8]:
np.random.seed(156)
# Chia data thành test và train với tỉ lệ test_size
df_train, df_test = train_test_split(df,test_size=0.25)
# Gọi class Naive Bayes để hỗ trợ, với thresh = 0.75 - ngưỡng kích hoạt yes/no
myClassifier = Binary_Naive_Bayes_Classifier(thresh=0.75)
# Build model dựa trên tập train và xuất ra độ chính xác trên tập training
myClassifier.fit(df_train)
# Dự đoán dựa trên tập data (có nhãn) và xuất ra độ chính xác trên tập test
_ = myClassifier.evaluate(df_test)

Training accuracy: 77.0 %
Test accuracy: 80.0 %


Tuyệt vời! Vậy là model chúng ta đã cải thiện `test accuracy` từ 75% lên 80% rồi đấy!